In [1]:
from pathlib import Path
import pandas as pd
from rich import print as rprint
import sqlite3
import sqlalchemy as sa
from sqlalchemy import text

In [5]:
DATA_CSV   = Path("data/reviews_analyzed.csv")
DB_FILE    = Path("db/bank_reviews.db")
DB_FILE.parent.mkdir(exist_ok=True)   # make sure directory exists


In [6]:
# SQLAlchemy connection string for SQLite
engine = sa.create_engine(f"sqlite:///{DB_FILE}")

# %% 🏗️  Build (or confirm) schema
with engine.begin() as conn:
    conn.exec_driver_sql("""
        CREATE TABLE IF NOT EXISTS banks (
            bank_id   INTEGER PRIMARY KEY AUTOINCREMENT,
            bank_name TEXT UNIQUE NOT NULL
        );
    """)
    conn.exec_driver_sql("""
        CREATE TABLE IF NOT EXISTS reviews (
            review_id       INTEGER PRIMARY KEY AUTOINCREMENT,
            bank_id         INTEGER,
            review_text     TEXT NOT NULL,
            rating          INTEGER NOT NULL,
            review_date     DATE   NOT NULL,
            sentiment_label TEXT,
            sentiment_score REAL,
            theme           TEXT,
            source          TEXT,
            FOREIGN KEY (bank_id) REFERENCES banks(bank_id)
        );
    """)
rprint("[green]Schema ensured in SQLite database.[/]")


Schema ensured in SQLite database.

In [7]:
# %% 📥  Load CSV
df = pd.read_csv(DATA_CSV, parse_dates=["date"])
rprint(f"[bold blue]Loaded[/] {len(df):,} rows from CSV")

# %% 🏦  Insert / upsert banks
with engine.begin() as conn:
    for bank in df["bank"].unique():
        conn.execute(text("""
            INSERT OR IGNORE INTO banks (bank_name) VALUES (:b)
        """), {"b": bank})

# Fetch bank_id lookup
bank_lookup = pd.read_sql("SELECT bank_id, bank_name FROM banks", engine)\
               .set_index("bank_name")["bank_id"].to_dict()

# %% 📤  Bulk‑insert reviews with pandas
df_out = (
    df.rename(columns={
        "review": "review_text",
        "date":   "review_date"
    })
    .assign(bank_id=lambda d: d["bank"].map(bank_lookup))
    .drop(columns=["bank"])
)


Loaded 986 rows from CSV